<a href="https://colab.research.google.com/github/MariaTerrah/Face-Mask-Detection-PyTorch/blob/master/TestPythonServier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I) Python et Data Engineering

In [44]:
import pandas as pd
import json


In [45]:


# Chargement des fichiers CSV
drugs_df = pd.read_csv('/content/drugs.csv')
clinical_trials_df = pd.read_csv('/content/clinical_trials.csv',encoding='utf-8')
pubmed_csv_df = pd.read_csv('/content/pubmed.csv')

# Chargement du fichier JSON
with open('/content/pubmed.json', 'r') as f:
    pubmed_json_df = pd.DataFrame(json.load(f))

# Visualisation des premières lignes
print("Drugs Data:")
print(drugs_df.head())

print("\nClinical Trials Data:")
print(clinical_trials_df.head())

print("\nPubMed CSV Data:")
print(pubmed_csv_df.head())

print("\nPubMed JSON Data:")
print(pubmed_json_df.head())


Drugs Data:
  atccode             drug
0   A04AD  DIPHENHYDRAMINE
1   S03AA     TETRACYCLINE
2   V03AB          ETHANOL
3   A03BA         ATROPINE
4   A01AD      EPINEPHRINE

Clinical Trials Data:
            id  \
0  NCT01967433   
1  NCT04189588   
2  NCT04237090   
3  NCT04237091   
4  NCT04153396   

                                                                                                                                                        scientific_title  \
0                                                                    Use of Diphenhydramine as an Adjunctive Sedative for Colonoscopy in Patients Chronically on Opioids   
1                                                                                   Phase 2 Study IV QUZYTTIR™ (Cetirizine Hydrochloride Injection) vs V Diphenhydramine   
2                                                                                                                                                                          
3  Fea

# Data cleansing

Dans cette étape de cleansing , l'idée est d'appliquer un ensemble de traitement pour nettoyer et uniformiser la donnée :

1.   Conversion en minuscule : comme Python est case-sensitive donc cette conversion est nécéssaire pour indiquer par exemple que DIPHENHYDRAMINE et diphenhydramine font référence à la même chose.
2.   Suppression des espaces non nécéssaires.
3.   Uniformisation du format des dates : les dates dans les fichiers ont plusieurs format différents ( 01/01/2019 , 2020-01-01 , "1 January 2020" ) donc il faut uniformiser le format pour pouvoir les traiter.
4. Gestion du mauvais encodage : exemple "Journal of emergency nursing\xc3\x28".





In [46]:


def format_date(df, date_column):
    df[date_column] = pd.to_datetime(df[date_column], errors='coerce', format=None)
    df[date_column] = df[date_column].dt.strftime('%Y-%m-%d')
    return df

clinical_trials_df = format_date(clinical_trials_df, 'date')
pubmed_csv_df = format_date(pubmed_csv_df, 'date')
pubmed_json_df = format_date(pubmed_json_df, 'date')



In [25]:

print("\nClinical Trials Data:")
print(clinical_trials_df.head())

print("\nPubMed CSV Data:")
print(pubmed_csv_df.head())

print("\nPubMed JSON Data:")
print(pubmed_json_df.head())


Clinical Trials Data:
            id  \
0  NCT01967433   
1  NCT04189588   
2  NCT04237090   
3  NCT04237091   
4  NCT04153396   

                                                                                                                                                        scientific_title  \
0                                                                    Use of Diphenhydramine as an Adjunctive Sedative for Colonoscopy in Patients Chronically on Opioids   
1                                                                                   Phase 2 Study IV QUZYTTIR™ (Cetirizine Hydrochloride Injection) vs V Diphenhydramine   
2                                                                                                                                                                          
3  Feasibility of a Randomized Controlled Clinical Trial Comparing the Use of Cetirizine to Replace Diphenhydramine in the Prevention of Reactions Related to Paclitaxel   
4       

In [48]:
# Fonction pour corriger les problèmes d'encodage dans les colonnes de texte
def correct_encoding(text):
    if isinstance(text, str):
        try:
            # Décoder les séquences d'échappement comme \\xc3\\x28 en UTF-8
            text = bytes(text, 'utf-8').decode('unicode_escape')
            # Corriger les séquences mal encodées
            text = text.encode('latin1').decode('utf-8', errors='ignore')
        except UnicodeDecodeError:
            pass  # En cas d'erreur, on laisse le texte inchangé
    return text

# Fonction pour appliquer le nettoyage sur les colonnes de texte des DataFrames
def clean_text_columns(df):
    text_columns = df.select_dtypes(include=['object']).columns
    for column in text_columns:
        df[column] = df[column].apply(correct_encoding)
    return df

# Appliquer la fonction de nettoyage sur les DataFrames
clinical_trials_df = clean_text_columns(clinical_trials_df)
pubmed_csv_df = clean_text_columns(pubmed_csv_df)
pubmed_json_df = clean_text_columns(pubmed_json_df)

# Définir les options pour afficher toutes les colonnes et le contenu complet des cellules
pd.set_option('display.max_colwidth', None)  # Pour afficher le contenu complet des cellules
pd.set_option('display.max_columns', None)   # Pour afficher toutes les colonnes
pd.set_option('display.max_rows', None)      # Pour afficher toutes les lignes (facultatif)

# Réafficher les DataFrames
print(clinical_trials_df)
print(pubmed_csv_df)
print(pubmed_json_df)


            id  \
0  NCT01967433   
1  NCT04189588   
2  NCT04237090   
3  NCT04237091   
4  NCT04153396   
5  NCT03490942   
6          NaN   
7  NCT04188184   

                                                                                                                                                        scientific_title  \
0                                                                    Use of Diphenhydramine as an Adjunctive Sedative for Colonoscopy in Patients Chronically on Opioids   
1                                                                                   Phase 2 Study IV QUZYTTIR™ (Cetirizine Hydrochloride Injection) vs V Diphenhydramine   
2                                                                                                                                                                          
3  Feasibility of a Randomized Controlled Clinical Trial Comparing the Use of Cetirizine to Replace Diphenhydramine in the Prevention of Reactions Re

In [27]:

print("\nClinical Trials Data:")
print(clinical_trials_df)

print("\nPubMed CSV Data:")
print(pubmed_csv_df)

print("\nPubMed JSON Data:")
print(pubmed_json_df)


Clinical Trials Data:
            id                                   scientific_title        date  \
0  NCT01967433  Use of Diphenhydramine as an Adjunctive Sedati...  2020-01-01   
1  NCT04189588  Phase 2 Study IV QUZYTTIR? (Cetirizine Hydroch...  2020-01-01   
2  NCT04237090                                                     2020-01-01   
3  NCT04237091  Feasibility of a Randomized Controlled Clinica...  2020-01-01   
4  NCT04153396  Preemptive Infiltration With Betamethasone and...  2020-01-01   
5  NCT03490942  Glucagon Infusion in T1D Patients With Recurre...         NaN   
6          NaN  Glucagon Infusion in T1D Patients With Recurre...         NaN   
7  NCT04188184  Tranexamic Acid Versus Epinephrine During Expl...  2020-04-27   

                                journal  
0          Journal of emergency nursing  
1          Journal of emergency nursing  
2          Journal of emergency nursing  
3          Journal of emergency nursing  
4       Hpitaux Universitaires de Gen

In [16]:
# Fonction pour nettoyer et normaliser les chaînes de caractères dans une colonne spécifique
def clean_text_column(column):
    """
    Nettoyer les chaînes de caractères dans une colonne de type texte.
    Appliquer uniquement aux éléments de type chaîne (texte) dans la colonne.
    """
    return column.apply(lambda x: x.lower().strip() if isinstance(x, str) else x)


In [17]:
# Appliquer clean_text_column sur des colonnes spécifiques dans clinical_trials_df
clinical_trials_df['scientific_title'] = clean_text_column(clinical_trials_df['scientific_title'])

# Appliquer clean_text_column sur la colonne 'title' dans pubmed_csv_df
pubmed_csv_df['title'] = clean_text_column(pubmed_csv_df['title'])

# Appliquer clean_text_column sur la colonne 'title' dans pubmed_json_df
pubmed_json_df['title'] = clean_text_column(pubmed_json_df['title'])

# Appliquer clean_text_column sur la colonne 'drug_name' dans drugs_df (exemple pour les médicaments)
drugs_df['drug_name'] = clean_text_column(drugs_df['drug_name'])


<ipython-input-17-520820783ae9>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_trials_df = clinical_trials_df.applymap(clean_text)
<ipython-input-17-520820783ae9>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pubmed_csv_df = pubmed_csv_df.applymap(clean_text)
<ipython-input-17-520820783ae9>:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pubmed_json_df = pubmed_json_df.applymap(clean_text)
<ipython-input-17-520820783ae9>:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  drugs_df = drugs_df.applymap(clean_text)


In [18]:
print("\nClinical Trials Data:")
print(clinical_trials_df)

print("\nPubMed CSV Data:")
print(pubmed_csv_df)

print("\nPubMed JSON Data:")
print(pubmed_json_df)

print("\Drugs Data:")
print(drugs_df)


Clinical Trials Data:
            id                                   scientific_title        date  \
0  NCT01967433  Use of Diphenhydramine as an Adjunctive Sedati...  2020-01-01   
1  NCT04189588  Phase 2 Study IV QUZYTTIR? (Cetirizine Hydroch...  2020-01-01   
2  NCT04237090                                                     2020-01-01   
3  NCT04237091  Feasibility of a Randomized Controlled Clinica...  2020-01-01   
4  NCT04153396  Preemptive Infiltration With Betamethasone and...  2020-01-01   
5  NCT03490942  Glucagon Infusion in T1D Patients With Recurre...         NaN   
6          NaN  Glucagon Infusion in T1D Patients With Recurre...         NaN   
7  NCT04188184  Tranexamic Acid Versus Epinephrine During Expl...  2020-04-27   

                                journal  
0          Journal of emergency nursing  
1          Journal of emergency nursing  
2          Journal of emergency nursing  
3          Journal of emergency nursing  
4       Hpitaux Universitaires de Gen

# Data pipeline


1.  Mention des médicaments


In [40]:
# Fonction pour trouver les mentions des médicaments dans les titres
def find_mentions(df, title, drugs):
    mentions = []
    for drug in drugs:
        matches = df[df[title].str.contains(drug, case=False, na=False)]
        for _, row in matches.iterrows():
            mentions.append({
                'drug': drug,
                'title': row[title],
                'journal': row['journal'],
                'date': row['date'],

            })
    return mentions


In [41]:
 #Obtenir la liste des médicaments à partir du fichier drugs.csv
drugs = drugs_df['drug'].tolist()

# Trouver les mentions dans les différents jeux de données
clinical_trials_mentions = find_mentions(clinical_trials_df, 'scientific_title', drugs)
pubmed_csv_mentions = find_mentions(pubmed_csv_df, 'title', drugs)
pubmed_json_mentions = find_mentions(pubmed_json_df, 'title', drugs)

# Combiner toutes les mentions dans un seul DataFrame
all_mentions = clinical_trials_mentions + pubmed_csv_mentions + pubmed_json_mentions
mentions_df = pd.DataFrame(all_mentions)

# Visualiser quelques lignes des mentions trouvées
print(mentions_df.head())

              drug  \
0  DIPHENHYDRAMINE   
1  DIPHENHYDRAMINE   
2  DIPHENHYDRAMINE   
3      EPINEPHRINE   
4    BETAMETHASONE   

                                                                                                                                                                   title  \
0                                                                    Use of Diphenhydramine as an Adjunctive Sedative for Colonoscopy in Patients Chronically on Opioids   
1                                                                                   Phase 2 Study IV QUZYTTIR™ (Cetirizine Hydrochloride Injection) vs V Diphenhydramine   
2  Feasibility of a Randomized Controlled Clinical Trial Comparing the Use of Cetirizine to Replace Diphenhydramine in the Prevention of Reactions Related to Paclitaxel   
3                                                                                                      Tranexamic Acid Versus Epinephrine During Exploratory Tympanotomy   
4      

In [51]:


# Fonction pour trouver les mentions de médicaments, générer le fichier JSON, et afficher le contenu
def get_mentions_graph(df_list, titles, drugs_list,sources, output_file):
    mentions_graph = {}

    # Parcourir chaque DataFrame dans la liste des sources
    for df, title, source in zip(df_list, titles, sources):
        for drug in drugs_list:
            # Rechercher les mentions dans la colonne des titres spécifique à ce DataFrame
            matches = df[df[title].str.contains(drug, case=False, na=False)]
            for _, row in matches.iterrows():
                # Créer un nœud pour chaque médicament s'il n'existe pas encore
                if drug not in mentions_graph:
                    mentions_graph[drug] = []

                # Ajouter chaque mention comme une liaison avec journal, date et titre
                mentions_graph[drug].append({
                    'journal': row['journal'],
                    'date': row['date'],
                    'title': row[title],
                    'type': source
                })

    # Exporter directement le graphe en fichier JSON
    with open(output_file, 'w') as json_file:
        json.dump(mentions_graph, json_file, indent=4)

    # Afficher le contenu de mentions_graph
    print(json.dumps(mentions_graph, indent=4))

    print(f"Fichier JSON généré : {output_file}")



In [52]:

# Liste des médicaments
drug_names = drugs_df['drug'].tolist()

# Liste des DataFrames (Clinical Trials, PubMed CSV, PubMed JSON)
df_list = [clinical_trials_df, pubmed_csv_df, pubmed_json_df]

# Liste des colonnes de titre spécifiques à chaque DataFrame
titles = ['scientific_title', 'title', 'title']

sources = ['ClinicalTrials', 'PubMed', 'PubMed']

# Appeler la fonction pour générer directement le fichier JSON et afficher le graphe
get_mentions_graph(df_list, titles, drug_names,sources, 'mentions_graph_output.json')

{
    "DIPHENHYDRAMINE": [
        {
            "journal": "Journal of emergency nursing",
            "date": "2020-01-01",
            "title": "Use of Diphenhydramine as an Adjunctive Sedative for Colonoscopy in Patients Chronically on Opioids",
            "type": "ClinicalTrials"
        },
        {
            "journal": "Journal of emergency nursing",
            "date": "2020-01-01",
            "title": "Phase 2 Study IV QUZYTTIR\u2122 (Cetirizine Hydrochloride Injection) vs V Diphenhydramine",
            "type": "ClinicalTrials"
        },
        {
            "journal": "Journal of emergency nursing",
            "date": "2020-01-01",
            "title": "Feasibility of a Randomized Controlled Clinical Trial Comparing the Use of Cetirizine to Replace Diphenhydramine in the Prevention of Reactions Related to Paclitaxel",
            "type": "ClinicalTrials"
        },
        {
            "journal": "Journal of emergency nursing",
            "date": "2019-01-01",
     

# Traitement ad-hoc

Le nom du journal qui mentionne le plus de médicaments différents:


In [53]:


# Fonction pour trouver le journal qui mentionne le plus de médicaments différents
def journal_with_most_drug_mentions(mentions_graph):
    journal_dict = {}

    # Parcourir chaque médicament et ses mentions
    for drug, mentions in mentions_graph.items():
        for mention in mentions:
            journal = mention['journal']
            # Ajouter le journal au dictionnaire
            if journal not in journal_dict:
                journal_dict[journal] = set() #set pour unicité des éléments
            journal_dict[journal].add(drug)

    # Trouver le journal avec le plus de médicaments différents
    max_drugs_journal = max(journal_dict, key=lambda j: len(journal_dict[j]))
    return max_drugs_journal, len(journal_dict[max_drugs_journal])


In [54]:

with open('mentions_graph_output.json', 'r') as json_file:
    mentions_graph = json.load(json_file)

# Appel de la fonction pour trouver le journal avec le plus de mentions
journal, count = journal_with_most_drug_mentions(mentions_graph)
print(f"Le journal qui mentionne le plus de médicaments différents est {journal} avec {count} médicaments mentionnés.")

Le journal qui mentionne le plus de médicaments différents est The journal of maternal-fetal & neonatal medicine avec 2 médicaments mentionnés.


L’ensemble des médicaments mentionnés par les mêmes
journaux référencés par les publications scientifiques (PubMed) mais non les tests cliniques (Clinical
Trials):

In [55]:
def drugs_in_same_pubmed_journals(drug, mentions_graph):
    # Étape 1: Trouver les journaux dans lesquels le médicament donné est mentionné (seulement PubMed)
    pubmed_journals = set()

    # Parcourir les mentions pour le médicament donné
    if drug in mentions_graph:
        for mention in mentions_graph[drug]:
            # Ne prendre en compte que les mentions dans PubMed
            if mention['type'] == 'PubMed':
                pubmed_journals.add(mention['journal'])

    # Trouver les autres médicaments mentionnés dans ces journaux PubMed
    other_mentionned_drugs = set()

    # Parcourir les autres médicaments dans mentions_graph
    for other_drug, mentions in mentions_graph.items():
        if other_drug != drug:  # Exclure le médicament initial
            for mention in mentions_graph[other_drug]:
                # Vérifier si le médicament est mentionné dans les mêmes journaux, uniquement dans PubMed
                if mention['journal'] in pubmed_journals and mention['type'] == 'PubMed':
                    other_mentionned_drugs.add(other_drug)

    return other_mentionned_drugs


In [56]:
def load_mentions_graph(file_path):
    with open(file_path, 'r') as json_file:
        return json.load(json_file)

mentions_graph = load_mentions_graph('mentions_graph_output.json')


# Exemple d'application
drug = 'TETRACYCLINE'
other_mentionned_drugs = drugs_in_same_pubmed_journals(drug, mentions_graph)

print(f"Les médicaments mentionnés dans les mêmes journaux que {drug} (uniquement PubMed) sont : {other_mentionned_drugs}")


Les médicaments mentionnés dans les mêmes journaux que TETRACYCLINE (uniquement PubMed) sont : {'ETHANOL'}


# Pour aller plus loin


# II) SQL

Première partie du test:




```
SELECT date , SUM(prod_price*prod_qty) FROM TRANSACTIONS
WHERE date BETWEEN '2020-01-01' AND '2020-12-31'
GROUP BY date
ORDER BY date ASC;
```



Seconde partie du test



```
SELECT
  t.client_id,
  SUM(CASE WHEN pn.product_type = 'MEUBLE' THEN t.prod_price * t.prod_qty ELSE 0 END) AS ventes_meuble,
  SUM(CASE WHEN pn.product_type = 'DECO' THEN t.prod_price * t.prod_qty ELSE 0 END) AS ventes_deco
FROM
  TRANSACTIONS t
JOIN
  PRODUCT_NOMENCLATURE pn
ON
  t.prod_id = pn.product_id
GROUP BY
  t.client_id
;

```

